In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
import csv
#setting
datafile = 'studentperform.csv'
studentmodel = 'studentmodel.h5'
batch_size = 10
hidden_neuron = 10
trainsize = 900
iterasi = 200
def generatemodel(totvar):
# create and fit the LSTM network
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(3, batch_input_shape=(batch_size, totvar), activation='sigmoid'))
    model.add(tf.keras.layers.Dense(hidden_neuron, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
#read data
alldata = np.genfromtxt(datafile,delimiter=',')[1:]
#separate between training and test
trainparam = alldata[:900, :-1]
trainlabel = alldata[:900, -1]
testparam = alldata[900:, :-1]
testlabel = alldata[900:, -1]
trainparam = trainparam[len(trainparam)%10:]
trainlabel = trainlabel[len(trainlabel)%10:]
testparam = testparam[len(testparam)%10:]
testlabel = testlabel[len(testlabel)%10:]
###############
#normalization#
###############
trainparamnorm = np.zeros(np.shape(trainparam))
trainlabelnorm = np.zeros(np.shape(trainlabel))
testparamnorm = np.zeros(np.shape(testparam))
testlabelnorm = np.zeros(np.shape(testlabel))

for i in range(len(trainparam[0])-2):
    trainparamnorm[:,i] = (trainparam[:,i] - np.min(trainparam[:,i])) / (np.max(trainparam[:,i]) - np.min(trainparam[:,i]))
    testparamnorm[:,i] = (testparam[:,i] - np.min(trainparam[:,i])) / (np.max(trainparam[:,i]) - np.min(trainparam[:,i]))
for i in range(2):
    trainparamnorm[:,-2+i] = (trainparam[:,-2+i] - 0.0) / (20.0 - 0.0)
    testparamnorm[:,-2+i] = (testparam[:,-2+i] - 0.0) / (20.0 - 0.0)
#for label
trainlabelnorm = (trainlabel - np.min(trainlabel)) / (np.max(trainlabel) - np.min(trainlabel))
testlabelnorm = (testlabel - np.min(trainlabel)) / (np.max(trainlabel) - np.min(trainlabel))
######################
#build and save model#
######################
mod = generatemodel(len(trainparamnorm[0]))
mod.fit(trainparamnorm, trainlabelnorm, epochs=iterasi, batch_size=batch_size, verbose=2, shuffle=True)
#save trained model
mod.save(studentmodel)
G3pred = mod.predict(testparamnorm, batch_size=batch_size)
G3real = G3pred*20.0

errreal = mean_squared_error(testlabel, G3real)

print(errreal)
import pandas as pd
from sklearn import tree
import pydot
import graphviz
from sklearn.tree.export import export_graphviz
data = pd.read_csv('studentperform.csv')
clf = tree.DecisionTreeClassifier(criterion="gini")
y = np.zeros(G3real.shape[0])
for i in range(y.shape[0]):
    if G3real[i]>=np.mean(G3real):
        y[i]=1
    else:
        y[i]=0
clf.fit(testparamnorm,y)
print(clf.feature_importances_)
dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("Grade")
dot_data = export_graphviz(clf, out_file=None,feature_names=data.columns[1:8],
                           filled=True, rounded=True,
                           special_characters=True)
graph = graphviz.Source(dot_data)

print(data)


Train on 900 samples
Epoch 1/200
900/900 - 1s - loss: 0.0616
Epoch 2/200
900/900 - 0s - loss: 0.0364
Epoch 3/200
900/900 - 0s - loss: 0.0360
Epoch 4/200
900/900 - 0s - loss: 0.0354
Epoch 5/200
900/900 - 0s - loss: 0.0349
Epoch 6/200
900/900 - 0s - loss: 0.0345
Epoch 7/200
900/900 - 0s - loss: 0.0339
Epoch 8/200
900/900 - 0s - loss: 0.0331
Epoch 9/200
900/900 - 0s - loss: 0.0326
Epoch 10/200
900/900 - 0s - loss: 0.0320
Epoch 11/200
900/900 - 0s - loss: 0.0310
Epoch 12/200
900/900 - 0s - loss: 0.0307
Epoch 13/200
900/900 - 0s - loss: 0.0296
Epoch 14/200
900/900 - 0s - loss: 0.0285
Epoch 15/200
900/900 - 0s - loss: 0.0277
Epoch 16/200
900/900 - 0s - loss: 0.0266
Epoch 17/200
900/900 - 0s - loss: 0.0255
Epoch 18/200
900/900 - 0s - loss: 0.0246
Epoch 19/200
900/900 - 0s - loss: 0.0239
Epoch 20/200
900/900 - 0s - loss: 0.0228
Epoch 21/200
900/900 - 0s - loss: 0.0219
Epoch 22/200
900/900 - 0s - loss: 0.0211
Epoch 23/200
900/900 - 0s - loss: 0.0203
Epoch 24/200
900/900 - 0s - loss: 0.0197
Epoc

C:\Users\16127\Anaconda3\envs\tf\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.tree.export module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


      studytime  traveltime  failures  freetime  absences  G1  G2  G3
0             2           2         0         3         6   5   6   6
1             2           1         0         3         4   5   5   6
2             2           1         3         3        10   7   8  10
3             3           1         0         2         2  15  14  15
4             2           1         0         3         4   6  10  10
...         ...         ...       ...       ...       ...  ..  ..  ..
1039          3           1         1         4         4  10  11  10
1040          2           1         0         3         4  15  15  16
1041          2           2         0         1         6  11  12   9
1042          1           2         0         4         6  10  10  10
1043          1           3         0         4         4  10  11  11

[1044 rows x 8 columns]


In [2]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
reg = LinearRegression().fit(trainparamnorm, trainlabelnorm)
reg.predict(testparamnorm)*20
mean_squared_error(testlabel, G3real)
est = sm.OLS( trainlabelnorm, trainparamnorm)
est2 = est.fit()
print(est2.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.984
Model:                            OLS   Adj. R-squared (uncentered):              0.984
Method:                 Least Squares   F-statistic:                              7794.
Date:                Fri, 18 Dec 2020   Prob (F-statistic):                        0.00
Time:                        16:40:09   Log-Likelihood:                          1033.9
No. Observations:                 900   AIC:                                     -2054.
Df Residuals:                     893   BIC:                                     -2020.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

[[ True  True False ... False False  True]
 [ True  True False ... False False  True]
 [False False  True ...  True  True False]
 ...
 [ True  True False ... False False  True]
 [False False  True ...  True  True False]
 [ True  True False ... False False  True]]


3.3688959105934453

In [4]:
print(errreal)
import pandas as pd
from sklearn import tree
import pydot
import graphviz
from sklearn.tree.export import export_graphviz
data = pd.read_csv('studentperform.csv')
clf = tree.DecisionTreeClassifier(criterion="gini")
y = np.zeros(G3real.shape[0])
for i in range(y.shape[0]):
    if G3real[i]>=np.mean(G3real):
        y[i]=1
    else:
        y[i]=0
clf.fit(testparamnorm,y)
print(clf.feature_importances_)
dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("Grade")
dot_data = export_graphviz(clf, out_file=None,feature_names=data.columns[1:8],
                           filled=True, rounded=True,
                           special_characters=True)
graph = graphviz.Source(dot_data)


3.236449701126438
[0.         0.         0.         0.         0.00733949 0.03000652
 0.962654  ]
